<a href="https://colab.research.google.com/github/daltonbc96/prediccion-partos-prematuros/blob/main/predicci%C3%B3n_prematuridad_regression_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analisis preliminar del potencial de utilizacion de los datos del SIP para el entrenamiento de modelos de *machine learning*

El desarrollo de modelos de aprendizaje automático con datos del mundo real es una tarea desafiante con un gran potencial para contribuir a los retos a los que se enfrenta la sociedad. Concretamente, el área de la salud ha empezado a beneficiarse de los esfuerzos por desarrollar modelos predictivos dedicados a diversos fines, como el análisis de exploraciones por imagen, la estratificación del riesgo de la población o el descubrimiento de nuevos fármacos, entre otros. Si nos fijamos en el ámbito de la salud pública, es notoria la necesidad de realizar esfuerzos adicionales, de modo que podamos ampliar el uso de las tecnologías en este sentido. El potencial para ampliar el acceso a la atención sanitaria en la prevención de enfermedades y la promoción de la salud es enorme. Consciente de estos retos, la OPS, a través de su departamento de *Evidence and Intelligence for Action in Health* (EIH), conjuntamente con el *Latin American Center of Perinatology, Women and Reproductive Health* (CLAP/WR), están desarrollando un primer esfuerzo analítico para evaluar el potencial de utilizar los datos registrados en el Sistema de Información Perinatal (SIP) como fuente para entrenar algoritmos de aprendizaje automático sobre temas relacionados con la salud materno-infantil. La idea es permitir la formación de modelos que puedan utilizarse para mejorar la calidad de la atención prestada a mujeres y niños en la región de las Américas.
<br>
<br>
Para poder entrenar modelos de aprendizaje automático sobre datos estructurados brutos, es necesario realizar varios pasos analíticos asociados a las etapas de extracción, limpieza, tratamiento y modelización. Por lo general, para desarrollar modelos de datos estructurados ha resultado fructífera la siguiente secuencia de pasos:
<br>
<br>

|         **1.** Carga de Información del SIP y Procesamiento de Datos<br>
|         **2.** Análisis y Tratamiento de Datos Faltantes<br>
|         **3.** Controles de Asociación<br>
|         **4.** Evaluación de Modelos de *Machine Learning* para Problemas de Regresión<br>

<br>
<br>

Teniendo en cuenta los puntos ejemplificados anteriormente, este informe pretende detallar la secuencia de pasos, así como los códigos en lenguaje python que se pueden utilizar para cubrir los elementos necesarios, con el fin de permitir la evaluación de la potencialidad de los datos existentes con el SIP para la estructuración de modelos de aprendizaje automático relacionados con la salud materno-infantil. Como primera área de interés, los esfuerzos aquí detallados se dirigen al desarrollo de modelos destinados a la estratificación del riesgo de la población en cuanto a la ocurrencia de nacimientos prematuros. Nuestro objetivo es, mediante los datos disponibles en el SIP, estimar la semana probable de parto con los datos disponibles al inicio de la gestación. Esta información puede ayudar a los profesionales sanitarios de la región en el tratamiento de las embarazadas con alto potencial de parto prematuro. En las secciones siguientes se detallan los elementos necesarios, en términos de rutinas de tratamiento, para cubrir cada una de las 4 etapas definidas anteriormente.
<br>
<br>

## 1 - Carga de Información del SIP y Procesamiento de Datos

Esta primera etapa se enfoca en preparar el terreno para el análisis posterior, comenzando con la carga de las bibliotecas necesarias, los datos originales del SIP, y un diccionario de datos organizado en un archivo JSON. La importancia de esta fase radica en establecer una base sólida para el análisis de datos, asegurando que todas las herramientas y recursos necesarios estén disponibles y correctamente configurados. Este paso es crucial porque sentará las bases para todo el trabajo analítico posterior, garantizando la integridad y la accesibilidad de los datos.


###  1.1 Carga de Bibliotecas, Datos y Diccionario
En este bloque de código, se importan bibliotecas esenciales para el análisis de datos y el aprendizaje automático. Bibliotecas como Pandas y NumPy son fundamentales para la manipulación y análisis de datos. Sweetviz se utiliza para la visualización y análisis exploratorio, mientras que Scikit-NA, Datapane y Plotly Express ayudan en el tratamiento de datos faltantes y visualizaciones interactivas. Se emplea una librería específica, 'pregnant', que incluye funciones personalizadas para el procesamiento de datos.

El siguiente paso es la carga de los datos originales del SIP utilizando Pandas  y un archivo JSON que actúa como diccionario de datos. Este diccionario es vital para entender la estructura y el significado de los datos.

In [ ]:
!pip install sweetviz scikit_na unidecode datapane scikit-optimize shap

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append("/content/drive/MyDrive/Prematuros/personalFunctions")

In [ ]:
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
import sweetviz as sv
sv.config_parser.read("sweetviz_config.ini")
import scikit_na as na
import datapane as dp
import plotly.express as px
from pregnant import process_data, get_types, dummify_categorical_columns, rename_columns, missing_columns, plot_missing_data_and_unique_value, plot_corr_graph, identificar_columnas_no_numericas, graficar_matriz_correlacion, matriz_correlacion_optimizada
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import warnings
warnings.filterwarnings('ignore')

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.tree import DecisionTreeRegressor
from statsmodels.stats.outliers_influence import variance_inflation_factor

from matplotlib import pyplot as plt
import shap

from IPython.display import HTML

#df = pd.read_csv('/content/drive/MyDrive/Prematuros/Datos Originales/pregnancie.csv', low_memory=False)
dictionary = json.load(open("/content/drive/MyDrive/Prematuros/JSON Setup/dictionary_pregnancie.json","rb"))
#df

###  1.2 Exclusión de Variables sin Definición en el Diccionario

En esta etapa, se procede a eliminar del conjunto de datos las variables que no están definidas en el diccionario de datos proporcionado. Este paso es crucial para garantizar la consistencia y la relevancia de los análisis posteriores. Las variables indefinidas, como '0510', '0616', '0630', entre otras, son excluidas del conjunto de datos. Esta exclusión se basa en la premisa de que si una variable no está definida claramente, su contribución al análisis puede ser cuestionable o su interpretación ambigua, lo que podría afectar la calidad del modelo de aprendizaje automático.

In [ ]:
variables_indefinidas = ['0510','0616','0630','0634','0687','0747','0789','A059','A060','A061','A062','A069','A077','A078','A154',
                         'A157','A159','A160','A161','A162','A163','A173','A184','A193','A194','A195','A196','A197','A199','A216',
                         'A217','V022','V026','V027','V031','V032', 'V033','V034','V035','V036','V037','V038','V039','V040','V041',
                         'V042','V043','V044','V045','V046','V047','V048','V049','V050', 'V051','V052','V053','V054','V055','V056',
                         'V057','V058','V059','V060','V061','V062','V063','V064','V065','V151','V273','V274', 'V275','V276','V277',
                         'V278','V279','V280','_gestaId','_motherId']
df.drop(variables_indefinidas, inplace=True, axis=1)

###  1.3 Ajuste de Tipado de las Variables, Etiquetas y Renombramiento de Columnas

Se desarrolló una función de procesamiento de datos que recibe los datos y el diccionario de datos. Con base en las propiedades descritas en el diccionario, los datos se convierten en el formato correcto, pudiendo ser categórico, numérico, booleano, fecha y hora o cadena de texto. Además, se utiliza la información del diccionario para renombrar las variables, facilitando así su interpretación. El estándar usado es el nombre original de la variable seguido de un guion bajo y el significado de la variable, lo que ayuda a comprender mejor cada elemento del conjunto de datos.

In [ ]:
#processed_df, cols_in_data_not_in_json, cols_in_json_not_in_data = process_data(raw_data = df, json_dictionary = dictionary, check_columns_in_data_not_in_json = True, check_columns_in_json_not_in_data = True)
#processed_df = rename_columns(processed_df, dictionary)
#processed_df.to_parquet('/content/drive/MyDrive/Prematuros/processed_df.parquet.gzip')
processed_df = pd.read_parquet('/content/drive/MyDrive/Prematuros/processed_df.parquet.gzip')
processed_df

###  1.4 Análisis Descriptivo de los Datos Procesados

Se elaboró un informe de *Exploratory Data Analysis* (EDA) del conjunto de datos procesados antes de proceder a procesamientos más profundos que podrían alterar la estructura original de los datos. Este informe, disponible en formato HTML, se puede consultar en un archivo adjunto al cuaderno Jupyter. En él, es posible observar las características generales de cada variable, análisis de datos faltantes y visualizaciones gráficas. Este paso es fundamental para entender la naturaleza de los datos con los que se trabajará en las fases posteriores del modelado.

In [ ]:
#report = sv.analyze(processed_df, pairwise_analysis="off")
#report.show_html('/content/drive/MyDrive/Prematuros/processed_df_report.html', open_browser=False)
#report.show_notebook()
HTML(filename="/content/drive/MyDrive/Prematuros/processed_df_report.html")

## 2 - Análisis y Tratamiento de Datos Faltantes
En esta sección, nos enfocamos en el tratamiento y preparación de los datos para su posterior utilización en modelos de *machine learning*. Este proceso es fundamental para asegurar la calidad y la eficiencia de los modelos predictivos. Se abordan diversos aspectos como el análisis y tratamiento de datos faltantes, la remoción de información irrelevante y la transformación de variables categóricas, culminando con la creación de un reporte analítico detallado de los datos.

### 2.1 Análisis de Datos Faltantes
Es importante analizar y tratar los datos faltantes, ya que pueden influir significativamente en el rendimiento del modelo. Este análisis se divide en varias subsecciones para abordar diferentes aspectos de los datos faltantes.

#### 2.1.1 Análisis General de Datos Faltantes
Se realiza una revisión general de los datos faltantes en todo el conjunto de datos transformados. Este análisis proporciona una visión general del nivel de completitud de los datos y ayuda a identificar posibles problemas o áreas que requieren atención especial.

In [ ]:
missing_all_data = pd.DataFrame(na.summary(processed_df, per_column=False))
missing_all_data

####  2.1.2 Análisis Detallado por Columna


##### 2.1.2.1 Gráfico

Se presenta un gráfico de barras que muestra la distribución del porcentaje de datos faltantes en cada columna. Esta visualización es útil para identificar rápidamente las columnas con niveles altos de datos faltantes, lo que puede indicar la necesidad de un tratamiento adicional o la exclusión de ciertas variables.

In [ ]:
fig_missing, fig_unique = plot_missing_data_and_unique_value(processed_df, sort_descending=True, missing_threshold=70, figure_height=780)
fig_missing.show()
#fig_unique.show()

##### 2.1.2.2 Tabla
Esta tabla proporciona una visión más detallada de los valores faltantes en cada columna, complementando la información visual del gráfico y ofreciendo una perspectiva más profunda de los datos.

In [ ]:
missing_per_column = na.summary(processed_df)
dp.DataTable(missing_per_column.transpose())

####  2.1.3 Análisis de Datos Faltantes de la Variable Objetivo
Se analiza la variable objetivo en términos de valores faltantes y su distribución. Conocer la cantidad de datos faltantes y la naturaleza de la distribución de la variable objetivo es crucial para el desarrollo de modelos predictivos efectivos.

In [ ]:
processed_df["0198_edad_gestacional_al_parto"].hist()
missing_count = processed_df["0198_edad_gestacional_al_parto"].isnull().sum()
print(f"Número de valores missing: {missing_count}")
processed_df["0198_edad_gestacional_al_parto"].describe()

### 2.2 Remoción de Información y Tratamiento de Datos Faltantes

En esta sección, nos enfocaremos en la remoción de información innecesaria y en el tratamiento detallado de los datos faltantes. Este proceso es crucial para optimizar la calidad de los datos y garantizar la precisión en los modelos de *machine learning*.

####  2.2.1 Remoción de Informaciones Irrelevantes
Se identifican y eliminan variables y registros que no aportan valor o que pueden sesgar los resultados del modelo.

* **Pasos para la Remoción de Informaciones**:
Identificación de Columnas No Numéricas: Eliminamos aquellas columnas que no contienen datos numéricos, ya que nuestro modelo se centrará en variables cuantitativas.
* **Eliminación de Columnas con Alto Porcentaje de Valores Faltantes**: Las columnas con más del 70% de datos faltantes son eliminadas, considerando que su baja completitud puede afectar negativamente la calidad del modelo.
* **Limpieza de Filas Específicas**: Se remueven filas donde la 'edad gestacional al parto' es nula o anormalmente alta (>44 semanas), ya que estos datos podrían ser incorrectos o irrelevantes para el análisis. También eliminó los casos que faltaban de esta variable.

In [ ]:
#Columns
# Identificar las columnas no numéricas
non_numerical_columns = identificar_columnas_no_numericas(processed_df)

# Encontrar columnas con alto porcentaje de valores faltantes
cols_high_missing, cols_single_value, combined_cols = missing_columns(processed_df, missing_threshold=70)

# Combinar las listas de columnas a eliminar
columns_drop = list(set(cols_high_missing + non_numerical_columns))

# Eliminar las columnas especificadas de 'processed_df' para crear 'df_clean'
df_clean = processed_df.drop(columns=columns_drop)

# Rows
# Limpieza de filas en 'df_clean'
# Eliminar filas donde '0198_edad_gestacional_al_parto' es NaN
df_clean = df_clean[df_clean['0198_edad_gestacional_al_parto'].notna()]

# Eliminar filas donde '0198_edad_gestacional_al_parto' es mayor a 44 (semanas)
df_clean = df_clean[df_clean["0198_edad_gestacional_al_parto"] <= 44]
df_clean

####  2.2.2 Tratamiento de Datos Faltantes

El tratamiento de los datos faltantes se realiza con el fin de completar el conjunto de datos y prepararlo para el análisis predictivo. Se utilizan técnicas avanzadas para asegurar que el tratamiento de estos valores faltantes sea lo más preciso posible, sin introducir sesgos innecesarios. A continuación, se detalla cada paso del proceso.

##### Paso 1: Preparación para la Imputación

* **Exclusión de las Columnas Objetivo**: Se retiran temporalmente las columnas objetivo del conjunto de datos para evitar su alteración durante el proceso de imputación.
* **Marcado de Datos Faltantes en Columnas Numéricas**: Para cada columna numérica, se añade una nueva columna indicadora que marca si el valor original era faltante con '_was_missing'.
* **Selección de Columnas Categóricas**: Identificación de todas las columnas categóricas, excluyendo aquellas marcadas como '_was_missing'.
* **Transformación a Variables Dummy**: Conversión de variables categóricas en un formato adecuado para modelos de machine learning mediante la creación de variables binarias ('dummy variables') para cada categoría.


In [ ]:
# Excluir las columnas objetivo de las transformaciones
columnas_objetivo = ['0198_edad_gestacional_al_parto']
df_clean_excluido = df_clean.drop(columns=columnas_objetivo)

# Paso 1: Identificar las columnas numéricas y crear las columnas indicadoras
columnas_numericas = df_clean_excluido.select_dtypes(include=[np.number]).columns
for col in columnas_numericas:
    df_clean_excluido[col + '_was_missing'] = df_clean_excluido[col].isnull()

# Seleccionar solo las columnas categóricas originales, excluyendo las que tienen el sufixo '_was_missing'
columnas_categoricas = df_clean_excluido.select_dtypes(include=['object', 'category', 'boolean']).columns
columnas_categoricas = [col for col in columnas_categoricas if not col.endswith('_was_missing')]
df_clean_dummy = pd.get_dummies(df_clean_excluido, columns=columnas_categoricas, dummy_na=True, drop_first=True)
df_clean_dummy

##### Paso 2: Imputación de Datos Numéricos
Aplicación de MICE: Uso de la técnica de Imputación Múltiple por Cadena de Ecuaciones (MICE) para llenar los valores faltantes en las columnas numéricas.  Esta técnica es un enfoque avanzado para la imputación de datos faltantes que modela cada variable con valores faltantes como una función de otras variables y utiliza esa estimación para imputar los valores faltantes. La imputación se realiza de manera iterativa, lo que permite un tratamiento más sofisticado y preciso de los datos faltantes.

En este caso, se utiliza un **DecisionTreeRegressor** como estimador en el proceso de imputación, debido a su eficacia en el manejo de relaciones no lineales entre variables. El argumento **max_iter** especifica el número máximo de iteraciones de imputación, lo que permite un equilibrio entre precisión y eficiencia computacional. Tras la imputación, se verifica la presencia de datos faltantes para asegurar que el conjunto de datos esté completo.

In [ ]:
imputer = IterativeImputer(estimator=DecisionTreeRegressor(), max_iter=10, random_state=32, initial_strategy="median")
imputed_values = imputer.fit_transform(df_clean_dummy[columnas_numericas])

# Crear un DataFrame con los valores imputados para las columnas numéricas
df_imputed_numeric = pd.DataFrame(imputed_values, columns=columnas_numericas, index=df_clean.index)

##### Paso 3: Combinación de Datos Tratados

La siguiente información se combinó en un único dataframe:

* **Reintegración de Columnas Objetivo**: Se reincorporan las columnas objetivo al conjunto de datos.
* **Combinación con Datos Imputados y Dummificados**: Los datos numéricos imputados se combinan con las variables dummificadas y las columnas indicadoras.
* **Conversión de Tipos de Datos**: Todas las columnas se convierten al tipo float64 para mantener la consistencia en el conjunto de datos.
* **Eliminación de Columnas con Variación Nula**: Se identifican y eliminan las columnas que contienen un único valor, ya que no aportan información relevante para el modelo.

In [ ]:
#Combinar los resultados con las columnas dummizadas, las columnas indicadoras y las columnas objetivo
df_imputed = pd.concat([df_clean[columnas_objetivo], df_clean_dummy.drop(columnas_numericas, axis=1), df_imputed_numeric], axis=1)
# Convertendo todas as colunas para float64
df_imputed = df_imputed.astype('float64')

# Identificar colunas com apenas um valor único
colunas_sem_variacao = [col for col in df_imputed.columns if df_imputed[col].nunique() == 1]

# Remover essas colunas do DataFrame
df_imputed = df_imputed.drop(columns=colunas_sem_variacao)

df_imputed

In [ ]:
#df_imputed.to_parquet('/content/drive/MyDrive/Prematuros/df_imputed.parquet.gzip')
df_imputed = pd.read_parquet('/content/drive/MyDrive/Prematuros/df_imputed.parquet.gzip')
df_imputed

##### Paso 4: Verificación Final de Datos Faltantes
 Se realiza una última revisión para asegurar que no queden valores faltantes en el conjunto de datos. Esta verificación es crucial para confirmar que el proceso de tratamiento ha sido exitoso.

In [ ]:
missing_per_column = na.summary(df_imputed)
dp.DataTable(missing_per_column.transpose())

##### Paso 5: Reporte Analítico de los Datos Tratados

Se genera un reporte detallado de EDA para el conjunto de datos tratado. Este reporte proporciona una visión integral de los datos, incluyendo la distribución de las variables, la correlación entre ellas y otras estadísticas importantes. Este informe, disponible en formato HTML, se puede consultar en un archivo adjunto al cuaderno Jupyter.

In [ ]:
#report = sv.analyze(df_imputed, pairwise_analysis="off")
#report.show_html('/content/drive/MyDrive/Prematuros/df_imputed_report.html', open_browser=False)
#report.show_notebook()
HTML(filename="/content/drive/MyDrive/Prematuros/df_imputed_report.html")

## 3 - Controles de Asociación

Esta etapa es crucial para la preparación de los datos antes del modelado en machine learning. Aquí, se enfoca en evaluar y controlar las relaciones entre las variables, identificando correlaciones significativas con la variable objetivo y entre las variables predictoras. Este análisis es esencial para garantizar que el modelo final sea preciso y no esté sesgado por relaciones redundantes o irrelevantes.

### 3.1 - Cálculo de correlaciones basadas en la variable objetivo

Para investigar qué variables están asociadas con la variable objetivo, se realizó un análisis de correlación de Pearson. En este análisis, mantuvimos en la matriz de correlaciones sólo las variables que tenían una relación mínima de 0,1 o más con la variable objetivo. Este paso ayuda a identificar qué variables tienen una relación significativa con la variable objetivo, proporcionando una base sólida para seleccionar características relevantes para el modelo de aprendizaje automático. A continuación, estas correlaciones se visualizan en un mapa de calor, lo que facilita la interpretación y la selección de las variables predictoras más relevantes para el modelo.

In [ ]:
matriz_corr_abs = matriz_correlacion_optimizada(df_imputed, '0198_edad_gestacional_al_parto', 0.1, ignorar_columnas = [])
features_with_relation = matriz_corr_abs.dropna(how='all', axis=0).dropna(how='all', axis=1).columns

graficar_matriz_correlacion(matriz_corr_abs)

### 3.2 Análisis de Combinación Lineal

Se establece un umbral de correlación de 0.95 ou mas para identificar relaciones lineales perfectas entre las variables predictoras. El propósito de este análisis es identificar pares de variables que están tan fuertemente correlacionadas que efectivamente proporcionan la misma información al modelo. Se utiliza un gráfico tipo heatmap para visualizar estas correlaciones y facilitar la identificación de variables redundantes.

In [ ]:
filtered_matrix = matriz_corr_abs[((abs(matriz_corr_abs) >= 0.95))]
# Substituir valores na diagonal (autocorrelação) por NaN
np.fill_diagonal(filtered_matrix.values, np.nan)
# Contar o número de variáveis não totalmente missing (ignorando autocorrelações)
combo_linear_columns = filtered_matrix.dropna(how='all', axis=0).dropna(how='all', axis=1).columns
num_variaveis = len(combo_linear_columns)
print(f"Número de variables con problemas de Combo linear: {num_variaveis}")
graficar_matriz_correlacion(filtered_matrix)

### 3.3 Verificación de Multicolinearidad con VIF
La multicolinearidad es un fenómeno en el que las variables predictoras en un modelo estadístico están altamente correlacionadas. Esto puede llevar a problemas en la interpretación de los resultados y afectar la precisión del modelo. Para evaluar la multicolinearidad, se utiliza el Factor de Inflación de la Varianza (VIF, por sus siglas en inglés). El VIF mide cuánto se infla la varianza de un coeficiente estimado si tus variables predictoras están correlacionadas. Si el VIF es alto, significa que hay una fuerte correlación entre esa variable predictora y las demás.

En este proceso, se calcula el VIF para cada variable predictora, excluyendo la variable objetivo para no sesgar los resultados. La práctica estándar es revisar las variables con los VIF más altos, eliminar la más problemática y recalcular el VIF para el resto de las variables. Este proceso se repite hasta que todas las variables restantes tengan valores de VIF aceptables, generalmente un VIF menor a 5 o 10, dependiendo de los estándares utilizados.

In [ ]:
df_independent = df_imputed[features_with_relation].drop(['0198_edad_gestacional_al_parto'], axis=1)
vif2_data = pd.DataFrame()
vif2_data["Feature"] = df_independent.columns

# calculating VIF for each feature
vif2_data["VIF"] = [variance_inflation_factor(df_independent.values, i)
                          for i in range(len(df_independent.columns))]
vif2_data

### 3.4 Selección de Variables para Incorporar a los Modelos
Por último, se seleccionan las variables identificadas en la etapa anterior que muestran una asociación y se excluyen del conjunto de datos las que están perfectamente o casi perfectamente correlacionadas (combo lineal) y las que muestran multicolinealidad. Este paso es esencial para garantizar que el modelo no incluya información redundante, lo que podría afectar a su rendimiento.

In [ ]:
features_with_combolinear = []
features_with_multicolinealidad = []

features_with_relation_atualizada = [elemento for elemento in features_with_relation if elemento not in features_with_combolinear and elemento not in features_with_multicolinealidad]

## 4 - Evaluación de Modelos de Machine Learning para Problemas de Regresión
En esta sección, se evalúan diferentes modelos de *machine learning* específicamente diseñados para resolver problemas de regresión. El objetivo principal es predecir la semana de parto, un problema de regresión por su naturaleza continua. La importancia de esta sección radica en identificar el modelo que mejor prediga el momento del parto, lo cual es crucial para planificar intervenciones médicas adecuadas y mejorar los resultados de salud materno-infantil.

### 4.1 Separación de Características y Etiquetas
En esta parte, se procede a separar las variables del conjunto de datos en dos grupos: las variables predictoras (X) y la variable objetivo (Y), que en este caso es la semana de parto. Esta separación es un paso fundamental en la preparación de datos para el entrenamiento de modelos de machine learning.

In [ ]:
x = df_imputed[features_with_relation_atualizada].drop(['edad_gestacional_al_parto_binary', '0198_edad_gestacional_al_parto'], axis=1)
y = df_imputed["0198_edad_gestacional_al_parto"]

### 4.2 Selección de Algoritmos a Probar
Se definirán los algoritmos a probar, seleccionando un representante de cada una de las siguientes familias de algoritmos:
* **Algoritmos de Regresión**: Ideales para predecir variables continuas, como la semana de parto.
* **Algoritmos Basados en Instancias**: Útiles en casos donde las relaciones entre datos son más importantes que la estructura subyacente.
* **Algoritmos de Regularización**: Ayudan a prevenir el sobreajuste en modelos complejos.
* **Algoritmos Bayesianos**: Proporcionan estimaciones probabilísticas, útiles para entender la incertidumbre en las predicciones.
* **Redes Neuronales Artificiales**: Ofrecen flexibilidad y potencia para capturar relaciones complejas en los datos.
* **Algoritmos de Ensemble**: Combinan las fortalezas de varios modelos para mejorar el rendimiento general.




In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import BayesianRidge
from sklearn.neural_network import MLPRegressor


MLA = [
#Regression Algorithms
#LinearRegression(),

#Instance-based Algorithms
#SVR(),

#Regularization Algorithms
#ElasticNet(),

#Decision Tree Algorithms
#DecisionTreeRegressor(),

#Bayesian Algorithms
#BayesianRidge(),

#Artificial Neural Network Algorithms
#MLPRegressor(),

# Ensemble Algorithms
GradientBoostingRegressor()

]

### 4.3 Prueba de Diferentes Algoritmos de Machine Learning

En esta sección, abordamos la evaluación y comparación de diferentes algoritmos de *machine learning* para el problema de regresión, específicamente la predicción de la semana de parto. A continuación, se detalla cada paso del proceso, siguiendo el código proporcionado y centrando la atención en la optimización bayesiana de hiperparámetros con BayesSearchCV.

**Definición del Espacio de Hiperparámetros**

Cada algoritmo de *machine learning* tiene hiperparámetros específicos que influyen en su rendimiento. En esta fase, se define un "espacio de hiperparámetros" para cada algoritmo, que determina los posibles valores o rangos de valores que estos hiperparámetros pueden tomar. Por ejemplo:

- Para una Regresión Lineal (LinearRegression), no se requiere ajustar hiperparámetros esenciales.
- Para un SVR (Support Vector Regressor), se definen rangos para parámetros como 'C', 'epsilon' y el tipo de 'kernel'.
- Para otros algoritmos como ElasticNet, DecisionTreeRegressor, BayesianRidge, MLPRegressor y GradientBoostingRegressor, se establecen rangos específicos para sus respectivos hiperparámetros.

Estos espacios de hiperparámetros serán utilizados posteriormente en la optimización bayesiana mediante BayesSearchCV, buscando los valores óptimos que maximicen el rendimiento del modelo.
<br>
<br>
**Validación Interna de los Modelos Predictivos**

La validación interna de los modelos se realiza mediante un enfoque de validación cruzada basado en el nivel de participante, con dos divisiones (folds) y cinco repeticiones para la separación de los datos. Este método garantiza que cada muestra del conjunto de datos se utilice tanto en el entrenamiento como en la validación de los modelos, proporcionando una evaluación robusta y fiable del rendimiento del modelo.
<br>
<br>
**Proceso de Prueba y Optimización**

El proceso de prueba y optimización se lleva a cabo de la siguiente manera:
1. **Bucle de Validación Cruzada**: Se inicia un bucle para realizar la validación cruzada y la optimización bayesiana en cada repetición. Se utiliza KFold de scikit-learn para dividir los datos en conjuntos de entrenamiento y prueba.
2. **Optimización Bayesiana con BayesSearchCV**: Para cada algoritmo, se aplica BayesSearchCV, una técnica de optimización que utiliza un enfoque bayesiano para encontrar los mejores hiperparámetros. Este método es más eficiente que las técnicas tradicionales como la búsqueda en cuadrícula, ya que se adapta y aprende de cada iteración para mejorar la búsqueda. Esta técnica ajusta iterativamente los hiperparámetros basándose en resultados anteriores, lo que permite una búsqueda más eficiente y efectiva de los mejores parámetros para cada modelo.


3. **Evaluación y Comparación de Modelos**: Una vez entrenados y optimizados los modelos, se realizan predicciones en el conjunto de prueba. Se calculan métricas de rendimiento como el coeficiente de determinación (R2), el error absoluto medio (MAE) y el error cuadrático medio (RMSE). Estas métricas se registran en un DataFrame para comparar los diferentes algoritmos.
<br>
<br>
**Resultados**

Al final de este proceso, se obtiene un DataFrame que contiene el nombre del algoritmo, las métricas de rendimiento y los mejores hiperparámetros encontrados para cada uno. Esta comparativa permite identificar cuál de los algoritmos probados ofrece el mejor rendimiento en la tarea de predecir la semana de parto, tomando en cuenta tanto su precisión como su capacidad de generalización.

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical


# Supongamos que 'x' y 'y' son tus datos de entrada y salida.

# Definiendo el espacio de hiperparámetros para cada algoritmo.
param_spaces = {
   # 'LinearRegression': {},  # Não há hiperparâmetros essenciais a ajustar.
  #  'SVR': {'C': Real(0.1, 1000, "log-uniform"), 'epsilon': Real(0.01, 1), 'kernel': Categorical(['linear', 'poly', 'rbf', 'sigmoid'])},
 #   'ElasticNet': {'alpha': Real(0.0001, 1, "log-uniform"), 'l1_ratio': Real(0, 1)},
 #   'DecisionTreeRegressor': {'max_depth': Integer(1, 30), 'min_samples_split': Integer(2, 20)},
 #   'BayesianRidge': {'alpha_1': Real(1e-6, 1e+6, "log-uniform"), 'lambda_1': Real(1e-6, 1e+6, "log-uniform")},
 #   'MLPRegressor': {'hidden_layer_sizes': Integer(10, 200), 'alpha': Real(0.0001, 1, "log-uniform"), 'learning_rate_init': Real(0.001, 0.1, "log-uniform")},
    'GradientBoostingRegressor': {'n_estimators': Integer(100, 500), 'learning_rate': Real(0.01, 0.1), 'max_depth': Integer(3, 10)}
}


# Definiendo las columnas para el DataFrame que comparará los modelos.
MLA_columns = ['MLA Name', 'R2', 'Mean Absolute Perc Error', 'Mean Absolute Error', 'Root Mean Squared Error', 'Best Hyperparameters']
MLA_compare = pd.DataFrame(columns=MLA_columns)
row_index = 0

# Bucle para realizar la validación cruzada y la optimización bayesiana.
for _ in range(5):
    kf = KFold(2, shuffle=True)
    for train_index, test_index in kf.split(x):
        X_train, X_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        for alg in tqdm(MLA):
            alg_name = alg.__class__.__name__
                # Verifica se o algoritmo atual tem hiperparámetros definidos.
            if alg_name in param_spaces and param_spaces[alg_name]:
                # Aplica la optimización bayesiana para encontrar los mejores hiperparámetros.
                bayes_search = BayesSearchCV(alg, param_spaces[alg_name], n_iter=32, cv=2, n_jobs=-1, scoring='neg_mean_squared_error')
                bayes_search.fit(X_train, y_train)

                # Obteniendo el mejor modelo y sus hiperparámetros.
                best_model = bayes_search.best_estimator_
                best_params = bayes_search.best_params_
            else:
                # Entrena el modelo con los hiperparámetros por defecto.
                best_model = alg.fit(X_train, y_train)
                best_params = alg.get_params()

            # Realiza la predicción y calcula las métricas de rendimiento.
            predicted = best_model.predict(X_test)
            MLA_compare.loc[row_index, 'MLA Name'] = alg_name
            MLA_compare.loc[row_index, 'R2'] = round(r2_score(y_test, predicted), 2)
            MLA_compare.loc[row_index, 'Mean Absolute Perc Error'] = round(np.mean(np.abs((y_test - predicted) / predicted)), 2)
            MLA_compare.loc[row_index, 'Mean Absolute Error'] = round(mean_absolute_error(y_test, predicted), 2)
            MLA_compare.loc[row_index, 'Root Mean Squared Error'] = round(np.sqrt(mean_squared_error(y_test, predicted)), 2)
            MLA_compare.loc[row_index, 'Best Hyperparameters'] = str(best_params)
            row_index += 1

# Mostrando los resultados.
MLA_compare

#https://www.run.ai/guides/hyperparameter-tuning/bayesian-hyperparameter-optimization

### 4.4 Promedio de Métricas
Tras probar múltiples iteraciones de cada algoritmo, se calculará el promedio de las métricas de rendimiento para cada uno. Esto permitirá identificar qué algoritmo, en promedio, ofrece un mejor desempeño para predecir la semana de parto. Esta evaluación media es crucial para obtener una visión más equilibrada y consistente del rendimiento de los modelos, ya que considera múltiples ejecuciones en lugar de un único resultado.

In [ ]:
mean = MLA_compare.groupby(['MLA Name']).mean()
mean = mean[['R2', 'Mean Absolute Perc Error','Mean Absolute Error', 'Root Mean Squared Error']]
mean

### 4.5 Prueba del Mejor Algoritmo
Esta sección se enfoca en reproducir y evaluar el rendimiento del mejor modelo identificado en las pruebas anteriores. Se implementará el modelo con los hiperparámetros óptimos encontrados y se evaluará su capacidad predictiva.

#### 4.5.1 Separación de Entrenamiento y Pruebas
Se dividirán los datos en dos conjuntos: uno para el entrenamiento (70%) y otro para la prueba (30%). Esta división es esencial para evaluar cómo el modelo se desempeña con datos no vistos durante su entrenamiento, lo que proporciona una medida de su capacidad de generalización.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance

X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=.30,random_state=42)
print("Train size: ", len(X_train))
print("Test size: ", len(X_test))

#### 4.5.2 Construcción del Modelo
En esta fase, se reconstruye el mejor modelo utilizando los hiperparámetros identificados previamente. Este paso es crucial para confirmar que el modelo seleccionado es efectivo y robusto bajo las condiciones óptimas establecidas.

In [ ]:
personal_parameters = {
    'loss': 'squared_error',
    'learning_rate': 0.1,
    'n_estimators': 100,
    'subsample': 1.0,
    'criterion': 'friedman_mse',
    'min_samples_split': 2,
    'min_samples_leaf': 1,
    'min_weight_fraction_leaf': 0.0,
    'max_depth': 3,
    'min_impurity_decrease': 0.0,
    'init': None,
    'random_state': 42,
    'max_features': None,
    'alpha': 0.9,
    'verbose': 0,
    'max_leaf_nodes': None,
    'warm_start': False,
    'validation_fraction': 0.1,
    'n_iter_no_change': None,
    'tol': 0.0001,
    'ccp_alpha': 0.0
}



MLA = [
GradientBoostingRegressor(**personal_parameters)
]

MLA_columns = ['MLA Name', 'R2', 'Mean Absolute Perc Error', 'Mean Absolute Error', 'Root Mean Squared Error']
MLA_best = pd.DataFrame(columns = MLA_columns)

row_index = 0
for alg in MLA:
    model = alg.fit(X_train, y_train)
    predicted = model.predict(X_test)
    MLA_best.loc[row_index, 'MLA Name'] = alg.__class__.__name__
    MLA_best.loc[row_index, 'R2'] = round(r2_score(y_test, predicted), 2)
    MLA_best.loc[row_index, 'Mean Absolute Perc Error'] = round(np.mean(np.abs((y_test - predicted) / predicted)), 2)
    MLA_best.loc[row_index, 'Mean Absolute Error'] = round(mean_absolute_error(y_test, predicted), 2)
    MLA_best.loc[row_index, 'Root Mean Squared Error'] = round(np.sqrt(mean_squared_error(y_test, predicted)), 2)


#### 4.5.3 Métricas del Modelo
Se evaluarán las métricas del modelo utilizando el conjunto de prueba. Estas métricas proporcionarán una comprensión detallada del rendimiento del modelo en términos de precisión y capacidad de generalización.

In [ ]:
MLA_best

#### 4.5.4 Visualización de la Capacidad Predictiva del Modelo
Se realizará una visualización comparativa de los valores reales versus los valores predichos por el modelo. Esta comparación gráfica es clave para comprender visualmente cómo el modelo predice la semana de parto y dónde puede tener dificultades.

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(y_test, predicted, alpha=0.5, color='blue', label='Dados Preditos')
plt.plot(y_test, y_test, color='red', label='Linha Ideal (Dados Reais)')
plt.title('Comparación de las semanas reales con las estimadas')
plt.xlabel('Semanas Estimadas')
plt.ylabel('Semana Reales')
plt.legend()
plt.show()

#### 4.5.5 Visualización del Error Predictivo
Se mostrará la distribución del error de predicción. Idealmente, esta distribución debería estar concentrada cerca de cero, con la menor dispersión posible en el eje Y, indicando errores pequeños y consistentes.

In [ ]:
sem_error = y_test - predicted
sem_error.hist(bins=30)
plt.title('Prediction Error Histogram')

#### 4.5.6 Importancia de las Características
Esta sección abordará la influencia de cada variable predictora en el modelo. Se utilizará el concepto de "importancia de las características" para identificar qué variables tienen un mayor impacto en las predicciones. Un valor más alto en la importancia de la característica indica una mayor influencia en el rendimiento predictivo del modelo. Esta información es crucial para entender qué factores son más relevantes a la hora de predecir la semana de parto.

In [ ]:
feature_importance = model.feature_importances_
feature_importance = feature_importance[feature_importance > 0.005]
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0])
fig = plt.figure(figsize=(12, 6))
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, np.array(x.columns)[sorted_idx])
plt.title('Feature Importance')
fig.tight_layout()
plt.show()

#### 4.5.7 Comprendiendo el Modelo a un Nivel Más Profundo
Finalmente, se utilizará el análisis SHAP (Shapley Additive exPlanations) para comprender en detalle cómo cada característica influye en las predicciones del modelo. A través de gráficos de cascada (waterfall), se podrá visualizar no solo la importancia, sino también la dirección de la influencia (aumentando o disminuyendo la predicción) de cada variable.

**¿Qué es SHAP y Cómo Funciona?**

SHAP es un enfoque de teoría de juegos para explicar el resultado de cualquier modelo de machine learning. Proporciona valores que explican el impacto de tener una cierta característica en la predicción comparada con la predicción promedio del modelo. Estos valores pueden ser positivos o negativos, dependiendo de si la característica aumenta o disminuye la predicción.

**Interpretación de Gráficos de Cascada (Waterfall) en SHAP**

Los gráficos de cascada muestran el impacto de cada característica en la predicción final. Comienzan con la predicción base (la media de todas las predicciones) y luego añaden el efecto de cada característica, mostrando cómo cada una contribuye a alejarse de la predicción base hacia la predicción final. Esto permite una comprensión visual intuitiva de cómo y por qué el modelo está haciendo sus predicciones, facilitando la interpretación y la toma de decisiones basadas en el modelo.

Más información:  https://www.aidancooper.co.uk/a-non-technical-guide-to-interpreting-shap-analyses/

In [ ]:
# Calcular os valores SHAP
explainer = shap.Explainer(model, X_train)
shap_values = explainer(X_test)

#shap.summary_plot(shap_values, X_test)
shap.plots.waterfall(shap_values[0])
